# Transformer Block

The Transformer block is a key component of the Transformer model, which was introduced in the paper "Attention is All You Need" by Vaswani et al.

The Transformer block consists of two main parts:

1. **Multi-Head Self-Attention Mechanism**: This mechanism allows the model to focus on different positions of the input sequence when producing the output. It does this by applying the attention mechanism multiple times in parallel (hence "multi-head") to the input. The attention scores are calculated using the formula:

    $$ \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V $$

    where $Q$, $K$, and $V$ are the query, key, and value vectors, and $d_k$ is the dimension of the key vectors.

2. **Position-wise Feed-Forward Networks**: These are fully connected feed-forward networks applied to each position separately and identically. This consists of two linear transformations with a ReLU activation in between.

The output of each sub-layer (Multi-Head Attention and Feed-Forward) is then passed through a residual connection followed by layer normalization.

The Transformer block can be visualized as follows:

<div style="text-align: center;">
    <img src="imgs/transformer.png" alt="Transformer Block Diagram" width="300">
</div>

This architecture allows the Transformer to handle dependencies regardless of their distance in the input or output sequences, making it effective for a wide range of tasks.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from transformer_from_scratch import MultiHeadAttention, AddNorm, FeedForwardNetwork

In [ ]:
d_model = 4
heads = 2
seq_length = 3
batch_size = 10

In [ ]:
# Assume we have a MultiheadAttention with some input data
input_data = torch.rand(batch_size, seq_length, d_model)
mha = MultiHeadAttention(d_model=d_model, heads=heads)

In [ ]:
# Clone the input into the query, key, and value tensors
query = input_data.clone()
key = input_data.clone()
value = input_data.clone()

# Pass the input data through the MultiheadAttention
output = mha(query, key, value)

In [ ]:
# Create a figure with 1 row and 2 columns
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

# Visualize the first element of the input batch
sns.heatmap(input_data[0].detach().numpy(), ax=axs[0], cmap="viridis")
axs[0].set_title("First element of the input batch")

# Visualize the output of the SelfAttention layer (first element of the output batch)
sns.heatmap(output[0].detach().numpy(), ax=axs[1], cmap="viridis")
axs[1].set_title("Output of SelfAttention for the first element of the batch")

# Set x and y aspect to "equal" so that each cell is square-shaped
axs[0].set_aspect("equal")
axs[1].set_aspect("equal")

# Display the figure
plt.show()

# Add & Norm

The Add & Norm (also known as residual connection and layer normalization) is a key component in the Transformer block.

1. **Residual Connection**: This is the "add" part. For each sub-layer (Multi-Head Attention and Feed-Forward), the input is added to the output of the sub-layer. This can be represented as:

    $$ \text{SubLayer}(x) = x + \text{MultiHead}(x) $$

    or

    $$ \text{SubLayer}(x) = x + \text{FeedForward}(x) $$

    where $x$ is the input to the sub-layer, and $\text{MultiHead}(x)$ or $\text{FeedForward}(x)$ is the output of the sub-layer. This allows the model to bypass the sub-layers during training, which helps to mitigate the vanishing gradient problem.

2. **Layer Normalization**: This is the "norm" part. After the residual connection, layer normalization is applied. This is a normalization technique that transforms the outputs to have a mean of 0 and a standard deviation of 1. This helps to stabilize the training process and reduces the training time. The layer normalization can be represented as:

    $$ \text{LayerNorm}(x) = \frac{x - \text{mean}(x)}{\sqrt{\text{var}(x) + \epsilon}} $$

    where $\text{mean}(x)$ is the mean of $x$, $\text{var}(x)$ is the variance of $x$, and $\epsilon$ is a small number to prevent division by zero.

Together, the Add & Norm can be represented as:

$$ \text{SubLayer}(x) = \text{LayerNorm}(x + \text{MultiHead}(x)) $$

or

$$ \text{SubLayer}(x) = \text{LayerNorm}(x + \text{FeedForward}(x)) $$

This architecture allows the Transformer to learn complex patterns in the data while maintaining a stable training process.

In [ ]:
# Assume we have an AddNorm instance
add_norm = AddNorm(d_model=4)

# Forward pass through the AddNorm layer
output_add_norm = add_norm(input_data, output)

# Calculate the global minimum and maximum
vmin = min(input_data.min(), output.min(), output_add_norm.min())
vmax = max(input_data.max(), output.max(), output_add_norm.max())

# Create a figure with 1 row and 3 columns
fig, axs = plt.subplots(1, 3, figsize=(30, 10))

# Visualize the first element of the input batch
sns.heatmap(input_data[0].detach().numpy(), ax=axs[0], cmap="viridis", vmin=vmin, vmax=vmax)
axs[0].set_title("First element of the input batch")

# Visualize the output of the SelfAttention layer (first element of the output batch)
sns.heatmap(output[0].detach().numpy(), ax=axs[1], cmap="viridis", vmin=vmin, vmax=vmax)
axs[1].set_title("Output of SelfAttention for the first element of the batch")

# Visualize the output of the AddNorm layer (first element of the output batch)
sns.heatmap(output_add_norm[0].detach().numpy(), ax=axs[2], cmap="viridis", vmin=vmin, vmax=vmax)
axs[2].set_title("Output of AddNorm for the first element of the batch")

# Set x and y aspect to "equal" so that each cell is square-shaped
axs[0].set_aspect("equal")
axs[1].set_aspect("equal")
axs[2].set_aspect("equal")

# Display the figure
plt.show()

### Fully Connected Layer (Feed-Forward Layer)

The Fully Connected Layer, also known as the Feed-Forward Layer, is a key component in the Transformer block. It follows the first normalization layer (Add & Norm).

The Fully Connected Layer consists of two linear transformations with a ReLU activation in between. It can be represented as:

$$ \text{FFN}(x) = \text{max}(0, xW_1 + b_1)W_2 + b_2 $$

where $x$ is the input to the layer, $W_1$ and $W_2$ are the weight matrices, $b_1$ and $b_2$ are the bias vectors, and $\text{max}(0, \cdot)$ is the ReLU activation function.

The Fully Connected Layer is applied to each position separately and identically. This means that the same layer is applied to each position in the sequence independently. This allows the model to learn complex patterns in the data while maintaining a stable training process.

The output of the Fully Connected Layer is then passed through another residual connection followed by layer normalization, which can be represented as:

$$ \text{SubLayer}(x) = \text{LayerNorm}(x + \text{FFN}(x)) $$

This architecture allows the Transformer to handle dependencies regardless of their distance in the input or output sequences, making it effective for a wide range of tasks.

The Feed-Forward Network (FFN) in the Transformer model consists of two linear layers. The first linear layer expands the dimensionality of the input, and the second linear layer reduces it back to the original dimensionality. 

If the input to the FFN has a shape of `(batch_size, seq_len, d_model)`, where `d_model` is the dimensionality of the input (also known as the number of features or the embedding size), then the shapes inside the FFN would be as follows:

1. Input: `(batch_size, seq_len, d_model)`
2. After the first linear layer (expansion): `(batch_size, seq_len, d_ff)`, where `d_ff` is typically larger than `d_model`.
3. After ReLU activation: `(batch_size, seq_len, d_ff)`
4. After the second linear layer (reduction): `(batch_size, seq_len, d_model)`

So, the output of the FFN has the same shape as the input: `(batch_size, seq_len, d_model)`. This allows it to be added to the input in the subsequent residual connection.

In [ ]:
# Assume we have a Feed-Forward Network (fully connected layer) instance and a second AddNorm instance
ffn = FeedForwardNetwork(d_model=4, d_ff=16)
add_norm_2 = AddNorm(d_model=4)

# Forward pass through the Feed-Forward Network
output_ffn = ffn(output_add_norm)

# Forward pass through the second AddNorm layer
output_add_norm_2 = add_norm_2(output_add_norm, output_ffn)

# Calculate the global minimum and maximum
vmin = min(output_add_norm.min(), output_ffn.min(), output_add_norm_2.min())
vmax = max(output_add_norm.max(), output_ffn.max(), output_add_norm_2.max())

# Create a figure with 1 row and 5 columns
fig, axs = plt.subplots(1, 3, figsize=(30, 10))

# Visualize the output of the first AddNorm layer (first element of the output batch)
sns.heatmap(output_add_norm[0].detach().numpy(), ax=axs[0], vmin=vmin, vmax=vmax, cmap="viridis")
axs[0].set_title("Output of first AddNorm for the first element of the batch")

# Visualize the output of the Feed-Forward Network (first element of the output batch)
sns.heatmap(output_ffn[0].detach().numpy(), ax=axs[1], vmin=vmin, vmax=vmax, cmap="viridis")
axs[1].set_title("Output of Feed-Forward Network for the first element of the batch")

# Visualize the output of the second AddNorm layer (first element of the output batch)
sns.heatmap(output_add_norm_2[0].detach().numpy(), ax=axs[2], vmin=vmin, vmax=vmax, cmap="viridis")
axs[2].set_title("Output of second AddNorm for the first element of the batch")

# Set x and y aspect to "equal" so that each cell is square-shaped
axs[0].set_aspect("equal")
axs[1].set_aspect("equal")
axs[2].set_aspect("equal")

# Display the figure
plt.show()

### Transformer Block

The Transformer block is a key component of the Transformer model, which is widely used in various natural language processing tasks. It consists of several steps:

1. **Multi-Head Self-Attention Mechanism**: This mechanism allows the model to focus on different positions of the input sequence when producing the output. It does this by applying the attention mechanism multiple times in parallel (hence "multi-head") to the input.

2. **Add & Norm (1st)**: The output of the Multi-Head Self-Attention Mechanism is then passed through a residual connection (the "add" part) followed by layer normalization (the "norm" part). This helps to stabilize the training process and reduces the training time.

3. **Position-wise Feed-Forward Networks**: These are fully connected feed-forward networks applied to each position separately and identically. This consists of two linear transformations with a ReLU activation in between.

4. **Add & Norm (2nd)**: The output of the Feed-Forward Networks is then passed through another residual connection followed by layer normalization.

The output of the Transformer block is a set of vectors. Each vector corresponds to a word in the input sequence, and its values represent the learned features of that word in the context of the entire sequence. This output can then be used in various ways depending on the specific task (e.g., for sequence classification, sequence generation, etc.).

The Transformer block is designed to handle dependencies regardless of their distance in the input or output sequences, making it effective for a wide range of tasks, especially those involving long sequences where long-term dependencies are important.